In [188]:
import s3fs
import shapefile # pyshp
import io
import pygeoif
import pyarrow as pa

In [42]:
client_kwargs = {
    "endpoint_url": "https://minio.minio-tenant",
    "aws_access_key_id": "admin",
    "aws_secret_access_key": "6bd71ace-8866-407a-9bcc-714bc5753f18",
    "verify": False
}

fs = s3fs.S3FileSystem(client_kwargs=client_kwargs)

In [152]:
version = sorted(fs.ls("s3://dutrajardim-fi/src/shapes"), reverse=True)[0].split('/')[-1]
adm_level = "2"
path = "s3://dutrajardim-fi/src/shapes/{version}/adm_{adm_level}/{{country}}/{version}_{{country}}_{adm_level}.{{ext}}".format(version=version, adm_level=adm_level)
adms = map(lambda path: path.split('/')[-1], fs.glob('s3://dutrajardim-fi/src/shapes/gadm40/adm_3/*'))

for adm in adms:
    sf = shapefile.Reader(
        shp=io.BytesIO(fs.open(path.format(country=adm, ext="shp")).read()),
        shx=io.BytesIO(fs.open(path.format(country=adm, ext="shx")).read()),
        dbf=io.BytesIO(fs.open(path.format(country=adm, ext="dbf")).read()),
        prj=io.BytesIO(fs.open(path.format(country=adm, ext="prj")).read()),
        cpg=io.BytesIO(fs.open(path.format(country=adm, ext="cpg")).read()),
    )


In [222]:
pa_records = pa.Table.from_pylist([
    {
        'geometry': pygeoif.as_shape(record.shape).wkt,
        'adm0': record.record[0],
        'adm1': '.'.join(record.record[1].split(".")[:2]) + "_1",
        'adm2': record.record[1],
        'name': record.record[2]
    } for record in sf.shapeRecords(fields=['ID_0', 'ID_2', 'NAME_2'])
])

In [228]:
pa_records.to_pandas()

,geometry,adm0,adm1,adm2,name
0,"POLYGON((-66.623916979473 -9.935029103147315, ...",BRA,BRA.1_1,BRA.1.1_1,Acrelândia
1,POLYGON((-69.42356109699995 -10.92717266099987...,BRA,BRA.1_1,BRA.1.2_1,Assis Brazil
2,POLYGON((-68.9998474119999 -11.006681441999888...,BRA,BRA.1_1,BRA.1.3_1,Brasiléia
3,POLYGON((-67.94033813399989 -9.337219236999886...,BRA,BRA.1_1,BRA.1.4_1,Bujari
4,POLYGON((-67.58827209399993 -10.51030063699988...,BRA,BRA.1_1,BRA.1.5_1,Capixaba
...,...,...,...,...,...
5499,POLYGON((-47.43127059899996 -6.066698072999941...,BRA,BRA.27_1,BRA.27.133_1,Tocantinópolis
5500,POLYGON((-48.14104461599993 -8.839973448999956...,BRA,BRA.27_1,BRA.27.134_1,Tupirama
5501,POLYGON((-48.28252029299995 -8.337325095999972...,BRA,BRA.27_1,BRA.27.135_1,Tupiratins
5502,POLYGON((-48.13484573299996 -7.123228073999883...,BRA,BRA.27_1,BRA.27.136_1,Wanderlândia
